# CH3-4

## TOC<a id='toc'></a>
* [Ch3 Notes](#ch3_notes)
* [Ch4 Notes](#ch4_notes)

### CH3 Notes <a id='ch3_notes'></a>
[toc](#toc)

### Generic Mapping Types
* good chedking if mapping: `isinstance(my_dict, abc.Mapping)`
    - better than just checking for dict to allow other maps
* all mapping types in the standard library use the basic `dict` in their implementation, so they all have to have *hashable* keys
* object is hashable if:
    - it has a hash value which never changes during its lifetime (needs a `__hash__()` method)
    - can be compared to other objects (need an `__eq__()` method)
    - and objects wich compare equal hash equal
* user defined types hash to `id()`, and all compare to not equal. If implement `__eq__()`, then only hashable if all attributes are immutable.

* Use *dictcomps*
* cool methods:
    - `d.get(k, [default])`
    - `d.__contains__(k)` called by `k in d`
        - careful when overwritting
    - `d.__missing__(k)` - called when `__getitem__` cannot find key (only in defaultdict)
    - `d.pop(k, [default])`
    - `d.setdefault(k, [default])` - if k in d, return d[k], otherwise set to default and return it
        - very efficient way to handle missing keys (particularly when value is mutable)
    - `d.update(m, [**kwargs])` - m can be mapping or iterable of (key, value) pairs
        * duck typing: first checks if has `keys` method. Otherwise falls back to iterating over m as tuples
        * <font color=red> What is the **kwargs for? </font>

In [20]:
class Temp:
    def __init__(self, x):
        self.x = x
        self.y = 3
        
    def __hash__(self):
        return 7
    
    def __eq__(self, other):
        return self.x == other.x

In [21]:
temp = Temp(4)
temp2 = Temp(5)

In [22]:
hash(temp), hash(temp2)

(7, 7)

In [13]:
temp == temp2

False

In [10]:
mydict = {1:2}

In [11]:
mydict.update([(3,4)])
mydict

{1: 2, 3: 4}

In [13]:
mydict.update({7:8}, new=10, old=11)
mydict

{1: 2, 3: 4, 7: 8, 'new': 10, 'old': 11}

* `collections.defaultdict` are also useful
     - default factory method only called in `__getitiem__`, not in others like `.get()`
     - uses the fact the `__getitem__` calls `__missing__` (if implemented) when missing key
* `collection.OrderedDict` allows iterating in predicatble fashion
* `collections.ChainMap` - holds list of mappings that can be searched as one
    - ex:   
    ```import builtins
    pylookup = ChainMap(locals(), globals(), vars(builtins))```
* `collections.Counter` - useful for counts; provides many convenience methods.
* `collections.UserDict` - pure python implementation of a mapping that works like a standard dict
    - designed to be subclassed; bult in dict has some implementation shortcuts that make it more difficult to subclass
    - doesnt inherit from dict, but contains typicall `__data__` dict

### Immutable Mappings
* `from types import MappingProxyType` - takes a dict and returns a view which cannot be modified
    - is dynamic; if underlying dict changes, proxy changes too.


In [46]:
from types import MappingProxyType

class oldClass:
    def __init__(self, some_dict):
        self.mutable_dict = some_dict
        
    def __hash__(self):
        return hash(self.mutable_dict)

class myClass:
    def __init__(self, some_dict):
        self.immutable_dict = MappingProxyType(some_dict)  
        
    def __hash__(self):
        return hash(self.immutable_dict)

In [47]:
obj = oldClass({'a':1, 'b':2})

In [48]:
hash(obj)

TypeError: unhashable type: 'dict'

In [49]:
obj = myClass({'a':1, 'b':2})

In [50]:
hash(obj)

TypeError: unhashable type: 'mappingproxy'

What a Shame.

In [51]:
class myHashClass:
    def __init__(self, some_dict):
        self.immutable_dict = frozenset(some_dict.items())  
        
    def __hash__(self):
        return hash(self.immutable_dict)

In [54]:
obj = myHashClass({'a':1, 'b':2})

In [55]:
hash(obj)

-6658507226492946828

### Set theory

* they are containers, iterables and sized - and then some
* infix notation: for sets `a` and `b`
    - union: `a | b`, intersection `a & b`, difference `a - b`
* literal syntax: `{1, 2, 3}` is faster than `set([1, 2, 3])` (calls specialize BUILD_SET bytecode)
* setComps use { and }, and no a:b
* Sets have many overloadable special methods for operator overloading
    - ex: `s & z` call `s.__and__(z)`, `s ^ z` calls `s.__xor__(z)`, etc.
* many predicate operators checking containments
     - nice method: `s.isdisjoint(z)`

### dict and set Under the Hood

* dict and set containment checks are extremely optimized
    - something like 4e-7 seconds per lookup, even for dict size of 1e7.
* source of their power, comes from the **hash table** implentation
    - hash table --> sparse array; cell often called **buckets**
    - in a dict hash table, there is a bucket for each item, with two fields (ref to key, ref to value)
    - all buckets have same size, easy to access by offset
    - python tries to keep at least 1/3 of buckets empty; if table grow to crowded, it is copie to a new location with more room
    - to put item, first calculate **hash value** of key
        * uses `hash()` builtin
        * works directly with builtin types, and calls `__hash__` for user defined (defaults to `id`)
* has table algo
    - to fetch `my_dict[search_key]` python calls `hash(search_key)`
    - it uses least significant bits of that result as offset to look up a bucket in hash table
        * number of bits depends on current size of table
    - if bucket empty, raise `KeyError`
    - if not empty: check `search_key == found_key`
    - if true, return found value
    - if bucekt not empty but keys don't match, this is a **hash collision**
    - if this happens, algo takes different bits in hash, massages them in a particular way, and uses result as an offset to look in a different bucket.
    - repeat process until KeyError or value match
* When inserting or updating
    - process to insert or update is the virtually same - except writes appropriately
        * when iserting, python may decide to rebuild hash table with more room
* things to note
    - average number of collisions per search is between one and two
    - dicts implementation trades space for time: they have significant memory overhead but provide fast access regardless of size
        * so if many records, don't store them as individual dicts (a-la JSON) but rather tuples
    - Key ordering depends on insertion
    - adding items to dict may change order of existing keys: 
    - ===> <font color=blue>VIP: Never iterate and modify at same time</font>
        * instead, iterate, collect, and then update.
    
          

<br>  
<hr>  
*Optimization is the altar where maintainability is sacrificed* 
<br>  
<hr>   

In [4]:
hash(1) == hash(1.0)

True

In [2]:
hash(0.1)

230584300921369408

In [49]:
bin(hash(0.1))

'0b1100110011001100110011001100110011001100110011001101000000'

In [52]:
bin(hash(1))

'0b1'

Interesting: starting with python 3.3, a *random salt value* is added to hashes of str, bytes and datatime objects
    - constant within process but varies between interpreter runs
    - security measure to prevent DOS attacks

Sets basically the same but hash bucket contain reference to only one element.

### CH4: Text vs Bytes <a id='ch4_notes'></a>
[toc](#toc)

* Handling of strings and characters represents a big difference in python 2 vs python 3
* string := sequence of characters
* **character** := ??
    - no longer true that 1 char == 1 byte
    - best defn: (abstract) set of elements idenfied as "U+" and four to six hex digits after it (called **code points**) - which map to letters/symbols/etc. 
        * Currently map is encoded in Unicode 6.3
        * ex: U+0041 --> A
        * only about 10% of possible combinations are actually mapped to characters
* The actual bytes that represent a character depend on the **encoding** in use.
    - map that convert conde points ot byte sequence
    - ex: UTF-8 sends A (U+0041) to single byte \x41, while UTF-16LE sends it to \x41\x00
* strings contain methods `.encode()` - pass something like 'utf8' -- results in `bytes` type
* can  `.decode()` bytes to string


In [62]:
mystr = "telaraña"

In [71]:
res = mystr.encode('utf8')
res

b'telara\xc3\xb1a'

In [72]:
type(res)

bytes

In [67]:
mystr.encode('utf16')

b'\xff\xfet\x00e\x00l\x00a\x00r\x00a\x00\xf1\x00a\x00'

In [70]:
b'\xc3\xb1'.decode('utf8')

'ñ'

### Byte Essentials
* `byte` is a new binary sequence type - immutable
* `bytarray` is mutable version of above
* each item is integer from 0 to 255
* thought really just ints, they are displayed using combination of ASCII, escape sequences, and hex by value
* `struct` module provides functions to parse packed bytes into a tuple of fields of different types, and go back to packed
    * if you work with binary data alot, learn `struct` and `mmap` module (memory-mapped file support)

In [87]:
temp = bytearray(2)

In [88]:
temp[0] = 0xc3
temp[1] = 0xb1

In [93]:
temp.decode('utf8')

'ñ'

In [95]:
byte_obj = bytes("telaraña", encoding='utf8')

In [99]:
byte_obj[:5]

b'telar'

In [100]:
type(byte_obj[:1]), type(byte_obj[0])

(bytes, int)

* The only squence for which `seq[0] == seq[:1]` is str.

In [102]:
bytes.fromhex('C3 B1').decode('utf8')

'ñ'

### Basic Encoders/Decoders

* Python distro comes with more than 100 *codecs* (encoder/decore) for text $<->$ byte conversions.
* `UnicodeEncodeError` - raised when character note defined in target encoding
* `UnicodeDecodeError` - when byte sequence leads does not represent any code point for given encoding
* UTF-8 is the default source encodingfor python 3 (ASCII was for python 2)
     - if load .py module with non UTF-8, get `SyntaxError` (unless other encoding specified - can add encoding on top of file)
* while can't know encoding without being told, `Chardet` package uses statistics of character appearances to guess at encoding
* some encodings start with a **BOM** (byte-order mark) to specify endianness of machine
    - UTF-16 encoding preprend text with ZERO WIDTH NO BREAK SPACE (U+FEFF)
    - if use UTF_16LE (explicitly little endian) or UTF-16BE (big endian), no such BOM is generated
    - this is no problem for UTF-8 because it is single byte - no ordering ambiguity
* best practice: bytes decoded to string as earlyas possible on input, and encode to bytes as late as possible as output
    - `open` built-in does decoding
    - careful with defautls (uses default locale encoding, so cross platform bugs)
    - called **unicode sandwich**

### Normalizing for string comparison


* challenge: more than one way to express same "string"
    - unicode has combining characters, like U+0301 COMBINING ACCUTE ACCENT
    - solution is to `unicodedata.normalize`
* **case folding** - convert all text to lower case (and then some)
    - `str.casefold()` (almost same as `str.lower()`)

### Other
* sorting can be a pain
* a lot of good info in unicode database
* different handling of str vs bytes in regular expression (and some other modules)
* In python 3, a str is stored in memory as a sequence of code points using a fixed number fo bytes per code point
    - but this is implementationd detail; and is flexible - and chooses width based on string

[toc](#toc)